In [1]:
import pineappl
import numpy as np
import pandas as pd

In [2]:
assert pineappl.__version__ == '0.8.7', "This only works with the mentioned PineAPPL version"

In [2]:
!pineappl convolve /Users/jaco/Documents/physics_projects/alphas_mtop/data/grids/40006002/ATLAS_TTBAR_13TEV_LJ_DIF_YTTBAR.pineappl.lz4 NNPDF40_nnlo_as_01180

b  atlas_yttbar   dsig/datlas_yttbar
  [atlas_yttbar]         [fb]       
-+-------+-------+------------------
0       0    0.25        5.5033866e5
1    0.25     0.5        5.1668547e5
2     0.5     0.8        4.7413921e5
3     0.8     1.1        3.9731078e5
4     1.1     1.4        3.2333928e5
5     1.4     1.8        1.9764636e5
6     1.8     2.5        8.0588827e4


In [3]:
def modify_bins(positive: bool = True) -> None:
    """Function to re-write the bin contents for the Grids."""
    grid = pineappl.grid.Grid.read("/Users/jaco/Documents/physics_projects/alphas_mtop/data/grids/40006002/ATLAS_TTBAR_13TEV_LJ_DIF_YTTBAR.pineappl.lz4")

    # Sign of the rapidity
    sign: int = 1 if positive else (-1)
    suffix: str = "positive" if positive else "negative"

    # Modify the bin limits
    modified_rap_bins = [
        (float(left), float(right))
        for left, right in zip(sign * grid.bin_left(0), sign * grid.bin_right(0))
    ]

    # Re-write the Grid bin limits
    remapper = pineappl.bin.BinRemapper(2.0 * grid.bin_normalizations(), modified_rap_bins)
    grid.set_remapper(remapper)
    grid.write_lz4(f"{suffix}-rapidity.pineappl.lz4")

    return

In [4]:
def check_bin_limits(positive: bool = True) -> None:
    suffix: str = "positive" if positive else "negative"
    grid = pineappl.grid.Grid.read(f"{suffix}-rapidity.pineappl.lz4")

    df = pd.DataFrame({"bin_left": grid.bin_left(0), "bin_right": grid.bin_right(0)})
    print(df)
    
    return

In [5]:
def reverse_bins(grid: pineappl.grid.Grid) -> None:
    """Reverse the bins for a given Grid."""
    n_bins = grid.bins()                # Get the number of bins
    n_orders = len(grid.orders())       # Get the number of PTOs
    n_channels = len(grid.channels())   # Get the number of Channels

    # Init a new Grid to store the reversed predictions
    reversed_bins = np.append(grid.bin_right(0)[-1], grid.bin_left(0)[::-1])
    new_grid = pineappl.grid.Grid.create(
        [pineappl.lumi.LumiEntry(c) for c in grid.channels()],
        grid.orders(),
        reversed_bins,
        pineappl.subgrid.SubgridParams()
    )

    # Collect the different subgrids at a given `boc`
    subgrids = {}
    inv_bindex = [i for i in range(n_bins)]
    for b in range(n_bins):
        for o in range(n_orders):
            for c in range(n_channels):
                bb = inv_bindex[-(b+1)]  # Get the reversed order of the bin index
                subgrid = grid.subgrid(o, bb, c)
                mu2_scales = [(mu2.ren, mu2.fac) for mu2 in subgrid.mu2_grid()]
                x1_grid = subgrid.x1_grid()
                x2_grid = subgrid.x2_grid()
                sub_array = subgrid.to_array3()

                subgrid_obj = pineappl.import_only_subgrid.ImportOnlySubgridV2(
                    array=sub_array,
                    mu2_grid=mu2_scales,
                    x1_grid=x1_grid,
                    x2_grid=x2_grid,
                )
                new_grid.set_subgrid(o, b, c, subgrid=subgrid_obj)

    # Fix back the normalization - Infer from the original Grid
    # TODO: Merge this with the function aboves!!!
    rap_bins_ref = [
        (float(left), float(right))
        for left, right in zip(new_grid.bin_left(0), new_grid.bin_right(0))
    ]
    remapper = pineappl.bin.BinRemapper(2.0 * new_grid.bin_normalizations(), rap_bins_ref)
    new_grid.set_remapper(remapper)
                
    new_grid.write_lz4("reversed-bins.pineappl.lz4")
    return

In [6]:
# Generate the negative rapidity bins
modify_bins(positive=False)

In [7]:
check_bin_limits(positive=False)

   bin_left  bin_right
0     -0.00      -0.25
1     -0.25      -0.50
2     -0.50      -0.80
3     -0.80      -1.10
4     -1.10      -1.40
5     -1.40      -1.80
6     -1.80      -2.50


In [8]:
# Generate the positive rapidity bins
modify_bins(positive=True)

In [10]:
check_bin_limits(positive=True)

   bin_left  bin_right
0      0.00       0.25
1      0.25       0.50
2      0.50       0.80
3      0.80       1.10
4      1.10       1.40
5      1.40       1.80
6      1.80       2.50


In [9]:
# Reverse the order of the Bins
reversed_bins_neg_rap = pineappl.grid.Grid.read("negative-rapidity.pineappl.lz4")
reverse_bins(grid=reversed_bins_neg_rap)

In [10]:
!pineappl convolve reversed-bins.pineappl.lz4 NNPDF40_nnlo_as_01180

b     x1         diff    
      []          []     
-+-----+-----+-----------
0  -2.5  -1.8 4.0294414e4
1  -1.8  -1.4 9.8823182e4
2  -1.4  -1.1 1.6166964e5
3  -1.1  -0.8 1.9865539e5
4  -0.8  -0.5 2.3706960e5
5  -0.5 -0.25 2.5834273e5
6 -0.25    -0 2.7516933e5


In [11]:
def merge_bins() -> None:
    """Merge two grids with different bins"""
    grid1 = pineappl.grid.Grid.read("./positive-rapidity.pineappl.lz4")
    grid2 = pineappl.grid.Grid.read("./reversed-bins.pineappl.lz4")
    grid2.merge(grid1)
    grid2.write_lz4("combined-rapidity.pineappl.lz4")

    return

In [12]:
merge_bins()

In [13]:
!pineappl convolve combined-rapidity.pineappl.lz4 NNPDF40_nnlo_as_01180

b      x1         diff    
       []          []     
--+-----+-----+-----------
 0  -2.5  -1.8 4.0294414e4
 1  -1.8  -1.4 9.8823182e4
 2  -1.4  -1.1 1.6166964e5
 3  -1.1  -0.8 1.9865539e5
 4  -0.8  -0.5 2.3706960e5
 5  -0.5 -0.25 2.5834273e5
 6 -0.25    -0 2.7516933e5
 7     0  0.25 2.7516933e5
 8  0.25   0.5 2.5834273e5
 9   0.5   0.8 2.3706960e5
10   0.8   1.1 1.9865539e5
11   1.1   1.4 1.6166964e5
12   1.4   1.8 9.8823182e4
13   1.8   2.5 4.0294414e4


In [14]:
2.7516933e5 * 2

550338.66